In [4]:
import numpy as np
import random, math
import copy

Dimension = 3
qubits = 2
A = np.array([[1, -1, 1], [-2, 3, 4], [1, -4, -1]]) # Row vectors
b = ([4, -4, 8]) # Column vectors

# 12 qubits with integer solutions
# x1 = 2 q0 + q1 - 2 q2 - q3
# x = {{1}, {-2}, {1}}
# QUBO minimum -96

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
                
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q|
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[ -34.   24.    0.    0.  -22.  -44.   22.   44.  -16.  -32.   16.   32.]
 [   0.  -56.    0.    0.  -44.  -88.   44.   88.  -32.  -64.   32.   64.]
 [   0.    0.   46.   24.   22.   44.  -22.  -44.   16.   32.  -16.  -32.]
 [   0.    0.    0.  104.   44.   88.  -44.  -88.   32.   64.  -32.  -64.]
 [   0.    0.    0.    0.  122.  104.    0.    0.   30.   60.  -30.  -60.]
 [   0.    0.    0.    0.    0.  296.    0.    0.   60.  120.  -60. -120.]
 [   0.    0.    0.    0.    0.    0.  -70.  104.  -30.  -60.   30.   60.]
 [   0.    0.    0.    0.    0.    0.    0.  -88.  -60. -120.   60.  120.]
 [   0.    0.    0.    0.    0.    0.    0.    0.   58.   72.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.  152.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  -22.   72.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   -8.]]

Minimum energy is  -96


Running code for D-Wave:

from dwave.system import DWaveSam

In [9]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-34.0, ('q2','q2'):-56.0, ('q3','q3'):46.0, ('q4','q4'):104.0, ('q5','q5'):122.0, ('q6','q6'):296.0, ('q7','q7'):-70.0, ('q8','q8'):-88.0, ('q9','q9'):58.0, ('q10','q10'):152.0, ('q11','q11'):-22.0, ('q12','q12'):-8.0}

quadratic = {('q1','q2'):24.0, ('q1','q5'):-22.0, ('q1','q6'):-44.0, ('q1','q7'):22.0, ('q1','q8'):44.0, ('q1','q9'):-16.0, ('q1','q10'):-32.0, ('q1','q11'):16.0, ('q1','q12'):32.0, ('q2','q5'):-44.0, ('q2','q6'):-88.0, ('q2','q7'):44.0, ('q2','q8'):88.0, ('q2','q9'):-32.0, ('q2','q10'):-64.0, ('q2','q11'):32.0, ('q2','q12'):64.0, ('q3','q4'):24.0, ('q3','q5'):22.0, ('q3','q6'):44.0, ('q3','q7'):-22.0, ('q3','q8'):-44.0, ('q3','q9'):16.0, ('q3','q10'):32.0, ('q3','q11'):-16.0, ('q3','q12'):-32.0, ('q4','q5'):44.0, ('q4','q6'):88.0, ('q4','q7'):-44.0, ('q4','q8'):-88.0, ('q4','q9'):32.0, ('q4','q10'):64.0, ('q4','q11'):-32.0, ('q4','q12'):-64.0, ('q5','q6'):104.0, ('q5','q9'):30.0, ('q5','q10'):60.0, ('q5','q11'):-30.0, ('q5','q12'):-60.0, ('q6','q9'):60.0, ('q6','q10'):120.0, ('q6','q11'):-60.0, ('q6','q12'):-120.0, ('q7','q8'):104.0, ('q7','q9'):-30.0, ('q7','q10'):-60.0, ('q7','q11'):30.0, ('q7','q12'):60.0, ('q8','q9'):-60.0, ('q8','q10'):-120.0, ('q8','q11'):60.0, ('q8','q12'):120.0, ('q9','q10'):72.0, ('q11','q12'):72.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc. chain_b.
1   1   0   0   0  0  0  0  0  0  0  1  1  -96.0      11      0.0
44  1   0   0   0  0  0  0  0  0  0  1  1  -96.0       1 0.083333
3   0   0   0   0  0  0  0  0  0  0  1  1  -90.0      16      0.0
4   1   0   0   0  1  0  0  0  0  1  0  1  -90.0       7      0.0
5   0   0   0   0  1  0  0  0  0  0  1  1  -90.0       1      0.0
6   0   0   0   0  0  1  0  0  0  1  1  1  -90.0     169      0.0
12  0   0   0   0  0  1  0  0  0  1  1  1  -90.0      93 0.083333
68  0   0   0   0  0  1  0  0  0  1  1  1  -90.0       1 0.166667
7   0   1   0   0  1  0  0  0  0  0  1  0  -88.0       5      0.0
8   0   0   0   0  0  0  0  0  0  0  1  0  -88.0       4      0.0
9   0   0   0   0  1  0  0  0  0  1  0  1  -86.0       8      0.0
10  0   0   0   0  0  0  0  0  0  1  1  1  -86.0      46      0.0
11  0   0   0   0  0  1  0  0  0  0  1  0  -86.0       7      0.0
19  0   0   0   0  0  0  0  0  0  1  1  1  -86.0      15 0.083333
13  0   0 

In [10]:
import dimod

#print(Q)
J = dimod.qubo_to_ising(Q)
print(J)

({'q1': -11.0, 'q2': -22.0, 'q3': 29.0, 'q4': 58.0, 'q5': 87.0, 'q6': 174.0, 'q7': -9.0, 'q8': -18.0, 'q9': 47.0, 'q10': 94.0, 'q11': 7.0, 'q12': 14.0}, {('q1', 'q2'): 6.0, ('q1', 'q5'): -5.5, ('q1', 'q6'): -11.0, ('q1', 'q7'): 5.5, ('q1', 'q8'): 11.0, ('q1', 'q9'): -4.0, ('q1', 'q10'): -8.0, ('q1', 'q11'): 4.0, ('q1', 'q12'): 8.0, ('q2', 'q5'): -11.0, ('q2', 'q6'): -22.0, ('q2', 'q7'): 11.0, ('q2', 'q8'): 22.0, ('q2', 'q9'): -8.0, ('q2', 'q10'): -16.0, ('q2', 'q11'): 8.0, ('q2', 'q12'): 16.0, ('q3', 'q4'): 6.0, ('q3', 'q5'): 5.5, ('q3', 'q6'): 11.0, ('q3', 'q7'): -5.5, ('q3', 'q8'): -11.0, ('q3', 'q9'): 4.0, ('q3', 'q10'): 8.0, ('q3', 'q11'): -4.0, ('q3', 'q12'): -8.0, ('q4', 'q5'): 11.0, ('q4', 'q6'): 22.0, ('q4', 'q7'): -11.0, ('q4', 'q8'): -22.0, ('q4', 'q9'): 8.0, ('q4', 'q10'): 16.0, ('q4', 'q11'): -8.0, ('q4', 'q12'): -16.0, ('q5', 'q6'): 26.0, ('q5', 'q9'): 7.5, ('q5', 'q10'): 15.0, ('q5', 'q11'): -7.5, ('q5', 'q12'): -15.0, ('q6', 'q9'): 15.0, ('q6', 'q10'): 30.0, ('q6', 'q11'

In [11]:
sampleset = sampler_auto.sample_ising(J[0], J[1], num_reads=100)
print(sampleset)
# max = 3457

   q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc. chain_.
0  +1  -1  -1  -1 -1 -1 -1 -1 -1 -1 +1 +1 -446.0       6     0.0
1  -1  -1  -1  -1 +1 -1 -1 -1 -1 -1 +1 +1 -440.0       1     0.0
2  -1  -1  -1  -1 -1 -1 -1 -1 -1 -1 +1 +1 -440.0       5     0.0
3  +1  -1  -1  -1 +1 -1 -1 -1 -1 +1 -1 +1 -440.0       3     0.0
4  -1  -1  -1  -1 -1 +1 -1 -1 -1 +1 +1 +1 -440.0       3     0.0
5  -1  +1  -1  -1 +1 -1 -1 -1 -1 -1 +1 -1 -438.0       4     0.0
6  -1  -1  -1  -1 -1 -1 -1 -1 -1 -1 +1 -1 -438.0       5     0.0
7  +1  +1  -1  -1 +1 -1 -1 -1 -1 -1 +1 -1 -436.0       5     0.0
8  -1  -1  -1  -1 -1 +1 -1 -1 -1 -1 +1 -1 -436.0       6     0.0
9  -1  -1  -1  -1 -1 -1 -1 -1 -1 +1 +1 +1 -436.0       5     0.0
10 +1  -1  -1  -1 -1 -1 -1 -1 -1 +1 -1 -1 -432.0       3     0.0
11 +1  +1  -1  -1 -1 -1 -1 -1 -1 +1 +1 -1 -432.0       7     0.0
12 -1  -1  -1  -1 +1 -1 -1 -1 -1 +1 -1 -1 -432.0       1     0.0
13 -1  -1  -1  -1 -1 -1 +1 -1 -1 +1 +1 -1 -432.0       2     0.0
14 -1  +1  -1  -1 -1 -1 -